In [1]:
#%%
from array import array
from cmath import nan
import statistics
from tkinter.ttk import Separator
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchviz import make_dot
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
import torchvision
import torchvision.transforms as transforms
from itertools import chain
from sklearn import metrics as met
import pickle
import icecream as ic
import torchsummary
import matplotlib.pyplot as plt
import pathlib
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from importlib import reload
import util
from torchmetrics import Accuracy
from torch.utils.data.datapipes.datapipe import _IterDataPipeSerializationWrapper, _MapDataPipeSerializationWrapper
from torch.utils.data import default_convert
from tqdm import tqdm
import time

# Rand seq

In [3]:

train_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/gene_seq_train.csv')
# train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data0/res_train.csv')

train_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/res_train.csv')
#don't touch test data, split out validation data from training data during training
test_data = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/gene_seq_test.csv')
# test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC/data0/res_test.csv')
test_target = pd.read_csv('/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/res_test.csv')

In [37]:
from random import choice
def rand_seq_gen(length):
   DNA=""
   for count in range(length):
      DNA+=choice("CGTA")
   return DNA

import random

# Example list
def rand_target(data_list = train_target.dropna().values.flatten()):
# Calculate frequencies
    elements, counts = np.unique(data_list, return_counts=True) 
    frequencies = counts / counts.sum()
    # Randomly sample from the list according to the distribution of elements
    sampled_element = random.choices(elements, weights=frequencies, k=13)        
    return sampled_element

In [38]:
#generating random sample in the training dataset
from random import seed
from random import randint
from tqdm import tqdm
p = 0.05 # <- what fraction of random seq do you want to add
rand_size = int(train_data.shape[0]*p)
random_df = pd.DataFrame(columns=[train_data.columns])
rand_res_df = pd.DataFrame(0, index=np.arange(rand_size), columns=[train_target.columns])

#get mean length of each loci
loci_len = {}
for x in train_data.columns:
    loci_len[x] = train_data[x].str.split("").str.len().mean().round()

#create instances with random sequences
target_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] #dr to be added in corresponding to rand seq

rand_size = int(len(train_data)*0.05) #amount of rand seq
train_data_rand = train_data.copy()
train_target_rand = train_target.copy()
seed(42)
# generate some integers for insert position
for _ in tqdm(range(rand_size)):
	insert_pos = randint(0, len(train_data))
	list_ = [] #creating rand seq
	for k, v in loci_len.items():
		list_.append(rand_seq_gen(int(v)))
	target_list = rand_target()
	train_data_rand = pd.DataFrame(np.insert(train_data_rand.values, insert_pos, list_, axis=0)) 
	train_target_rand = pd.DataFrame(np.insert(train_target_rand.values, insert_pos, target_list, axis=0)) 
	# for x in range(rand_size):
	# 	list_ = []
	# 	for k, v in loci_len.items():
	# 		list_.append(rand_seq_gen(int(v)))
	# 	random_df = random_df.append(pd.DataFrame([list_], columns=[train_data.columns]), ignore_index=True)
train_data_rand.columns = train_data.columns
train_target_rand.columns = train_target.columns

100%|██████████| 542/542 [00:26<00:00, 20.45it/s]


In [39]:

from pathlib import Path  
filepath = Path(f'/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/seq_{p*100}percent_rand.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
train_data_rand.to_csv(filepath, index=False, index_label=False)

filepath = Path(f'/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data1/res_{p*100}percent_rand.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
train_target_rand.to_csv(filepath, index=False, index_label=False)

# Rand for aa

In [2]:
data_aa = '/mnt/storageG1/lwang/Projects/tb_dr_MIC_v2/data_aa'
train_data = np.loadtxt(f'{data_aa}/aa_data_train.csv',delimiter =',')
test_data = np.loadtxt(f'{data_aa}/aa_data_test.csv',delimiter =',')
#don't touch test data, split out validation data from training data during training
train_target = pd.read_csv(f'{data_aa}/mic_aa_train.csv')
test_target = pd.read_csv(f'{data_aa}/mic_aa_test.csv')

In [6]:
train_data.shape

(10728, 27063)

In [4]:
train_data.shape

(10728, 27063)

In [5]:
def rand_target(data_list = train_target.dropna().values.flatten()):
# Calculate frequencies
    elements, counts = np.unique(data_list, return_counts=True) 
    frequencies = counts / counts.sum()
    # Randomly sample from the list according to the distribution of elements
    sampled_element = random.choices(elements, weights=frequencies, k=13)        
    return sampled_element

def rand_aa(data_list = train_data.flatten()):
# Calculate frequencies
    print('1')
    elements, counts = np.unique(data_list, return_counts=True) 
    frequencies = counts / counts.sum()
    print('2')
    # Randomly sample from the list according to the distribution of elements
    sampled_element = random.choices(elements, weights=frequencies, k=27063)
    print('3')       
    return sampled_element

In [7]:
elements, counts = np.unique(train_data.flatten(), return_counts=True) 
frequencies = counts / counts.sum()

In [14]:
elements

array([0., 1.])

In [13]:
counts

array([289985946,    345918])

In [16]:
import random

len(random.choices(elements, weights=frequencies, k=27063))

27063

In [20]:
#generating random sample in the training dataset
from random import seed
from random import randint
from tqdm import tqdm
p = 0.05 # <- what fraction of random seq do you want to add
rand_size = int(train_data.shape[0]*p)

#get mean length of each loci
# loci_len = {}
# for x in train_data.columns:
#     loci_len[x] = train_data[x].str.split("").str.len().mean().round()
# Calculate frequencies
elements, counts = np.unique(train_data.flatten(), return_counts=True) 
frequencies = counts / counts.sum()
# Randomly sample from the list according to the distribution of elements
#create instances with random sequences
target_list = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] #dr to be added in corresponding to rand seq


rand_size = int(len(train_data)*0.05) #amount of rand seq
train_data_rand = train_data.copy()
train_target_rand = train_target.copy()
seed(42)
# generate some integers for insert position
for _ in tqdm(range(rand_size)):
	insert_pos = randint(0, len(train_data))
	list_ = [] #creating rand seq
	list_ = train_data_rand
	target_list = rand_target()
	train_data_rand = np.insert(train_data_rand, insert_pos, list_, axis=0)
	train_target_rand = pd.DataFrame(np.insert(train_target_rand.values, insert_pos, target_list, axis=0)) 
	# for x in range(rand_size):
	# 	list_ = []
	# 	for k, v in loci_len.items():
	# 		list_.append(rand_seq_gen(int(v)))
	# 	random_df = random_df.append(pd.DataFrame([list_], columns=[train_data.columns]), ignore_index=True)


  1%|          | 3/536 [18:10<62:32:33, 422.43s/it]

In [18]:
len(train_data_rand)

NameError: name 'train_data_rand' is not defined

In [55]:
from pathlib import Path  
filepath = Path(f'/mnt/storageG1/lwang/Projects/tb_dr_MIC_2/data1/aa_{p*100}percent_rand.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
np.savetxt( filepath, train_data_rand, delimiter =',')

filepath = Path(f'/mnt/storageG1/lwang/Projects/tb_dr_MIC_2/data1/aa_res_{p*100}percent_rand.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)
train_target_rand.to_csv(filepath, index=False, index_label=False)